In [ ]:

#  Step 1: Create Patient Dataset with UNIQUE Diagnoses
import pandas as pd
import random, json
from google.colab import files

# Upload Mapping.csv
uploaded = files.upload()

# Load dataset
try:
    df_map = pd.read_csv("Mapping.csv")
except FileNotFoundError:
    print("Mapping.csv not found. Please upload it first.")
    # You might want to add a mechanism to prompt for upload or exit here

# Symptom mapping dictionary
symptom_dict = {
    "Normal": "No abnormal symptoms (routine scan)",
    "normal": "No abnormal symptoms (routine scan)",
    "Benign": "Headache, vision problems, nausea",
    "benign": "Headache, vision problems, nausea",
    "Malignant": "Severe headache, seizures, confusion",
    "malignant": "Severe headache, seizures, confusion",
    "Pneumonia": "Fever, cough, shortness of breath",
    "Squamous.cell.carcinoma left.hilum t1 n2 m0 iiia": "Persistent cough, chest pain, difficulty breathing",
    "Adenocarcinoma left.lower.lobe t2 n0 m0 ib": "Chest tightness, weight loss, chronic cough",
    "Stone": "Severe abdominal pain, painful urination",
    "Fractured": "Swelling, pain, difficulty in movement"
}

# Get UNIQUE diagnoses only
unique_diagnoses = df_map.drop_duplicates(subset=["Diagnosis"])

# Generate synthetic patients (one per unique diagnosis)
patients_list = []
for i, row in unique_diagnoses.iterrows():
    diag = row["Diagnosis"]
    patients_list.append({
        "Patient_ID": len(patients_list) + 1,
        "Age": random.randint(25, 75),
        "Gender": random.choice(["Male", "Female"]),
        "Symptoms": symptom_dict.get(diag, "General weakness, fatigue"),
        "Diagnosis": diag,
    })

# Convert to DataFrame
df_patients = pd.DataFrame(patients_list)

# Filter out patients with 'Unknown' ICD10_Code - This step is no longer needed as ICD10_Code is not included
# diagnoses_to_exclude = ['Stone', 'Fractured', 'Not fractured']
# df_patients = df_patients[~df_patients['Diagnosis'].isin(diagnoses_to_exclude)].reset_index(drop=True)

# Update Patient_ID after filtering
df_patients['Patient_ID'] = df_patients.index + 1

# Drop the 'ICD10_Code' column before saving and generating notes
if 'ICD10_Code' in df_patients.columns:
    df_patients = df_patients.drop(columns=['ICD10_Code'])


# Save patient data to CSV and JSON
csv_path = "/content/patients_unique.csv"
json_path = "/content/patients_unique.json"

df_patients.to_csv(csv_path, index=False)
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(df_patients.to_dict(orient='records'), f, indent=4)

print(f"Patient data saved to {csv_path} and {json_path}\n")

# Download the CSV and JSON files
files.download(csv_path)
files.download(json_path)



# Step 2: Generate Clinical Notes
# Plan mapping dictionary
plan_dict = {
    "Benign": "Schedule follow-up MRI in 6 months to monitor for changes. Advise on symptom management.",
    "benign": "Schedule follow-up MRI in 6 months to monitor for changes. Advise on symptom management.",
    "Malignant": "Refer to oncology for further evaluation and treatment planning. Order immediate CT scan.",
    "malignant": "Refer to oncology for further evaluation and treatment planning. Order immediate CT scan.",
    "Normal": "No specific follow-up required at this time. Continue routine check-ups.",
    "normal": "No specific follow-up required at this time. Continue routine check-ups.",
    "Pneumonia": "Prescribe antibiotics and rest. Schedule a follow-up chest X-ray in 2 weeks.",
    "Squamous.cell.carcinoma left.hilum t1 n2 m0 iiia": "Refer to thoracic surgery for consideration of resection. Order PET scan for staging.",
    "Adenocarcinoma left.lower.lobe t2 n0 m0 ib": "Discuss surgical options with patient. Schedule pulmonary function tests.",
    "General weakness, fatigue": "Order blood tests to investigate underlying causes of weakness and fatigue.", # Plan for diagnoses not explicitly mapped
}

# Assessment mapping dictionary
assessment_dict = {
    "Benign": "Based on the provided information, the patient presents with symptoms ({symptoms}) and imaging findings consistent with a benign condition. Further monitoring is recommended.",
    "benign": "Based on the provided information, the patient presents with symptoms ({symptoms}) and imaging findings consistent with a benign condition. Further monitoring is recommended.",
    "Malignant": "Based on the provided information, the patient presents with symptoms ({symptoms}) highly suggestive of a malignant condition. Urgent investigation and specialist referral are warranted.",
    "malignant": "Based on the provided information, the patient presents with symptoms ({symptoms}) highly suggestive of a malignant condition. Urgent investigation and specialist referral are warranted.",
    "Normal": "The patient presents with no reported abnormal symptoms ({symptoms}) and routine imaging findings are within normal limits. No immediate concerns identified.",
    "normal": "The patient presents with no reported abnormal symptoms ({symptoms}) and routine imaging findings are within normal limits. No immediate concerns identified.",
    "Pneumonia": "The patient presents with symptoms ({symptoms}) consistent with pneumonia. Imaging findings will help confirm the diagnosis and severity.",
    "Squamous.cell.cell carcinoma left.hilum t1 n2 m0 iiia": "The patient presents with symptoms ({symptoms}) indicative of squamous cell carcinoma. Staging investigations are required to determine the extent of the disease.",
    "Adenocarcinoma left.lower.lobe t2 n0 m0 ib": "The patient presents with symptoms ({symptoms}) potentially related to adenocarcinoma. Further evaluation is needed to confirm the diagnosis and plan treatment.",
    "General weakness, fatigue": "The patient reports general weakness and fatigue ({symptoms}). Further investigation is needed to determine the underlying cause.", # Assessment for diagnoses not explicitly mapped
}


def generate_clinical_note(patient):
    """Generates a clinical note for a given patient."""
    diagnosis = patient['Diagnosis']
    symptoms = patient['Symptoms']

    # Get assessment text based on diagnosis, with fallback to generic
    assessment_text_template = assessment_dict.get(diagnosis, assessment_dict.get("General weakness, fatigue"))
    assessment_text = assessment_text_template.format(symptoms=symptoms)


    # Get plan text based on diagnosis, with fallback to generic
    plan_text = plan_dict.get(diagnosis, plan_dict.get("General weakness, fatigue")) # Use generic plan if diagnosis not in dict

    note = f"""
Patient Name: Patient {patient['Patient_ID']}
Age: {patient['Age']}
Gender: {patient['Gender']}
Symptoms: {patient['Symptoms']}
Diagnosis: {patient['Diagnosis']}

Assessment: {assessment_text}

Plan: {plan_text}
""" # Removed "ICD-10 Code: {patient['ICD10_Code']}"
    return note.strip()

# Generate notes for each patient
clinical_notes = [generate_clinical_note(row) for index, row in df_patients.iterrows()]

# Display the first few notes
print("Generated Clinical Notes (Preview):\n")
for i, note in enumerate(clinical_notes[:3]):
    print(f"--- Note for Patient {i+1} ---")
    print(note)
    print("-" * 20)

# Save notes to a text file
notes_path = "/content/clinical_notes.txt"
with open(notes_path, "w", encoding="utf-8") as f:
    for note in clinical_notes:
        f.write(note)
        f.write("\n\n---\n\n") # Separator between notes

# Download the text file
files.download(notes_path)

print(f"Clinical notes saved to {notes_path} and downloaded.")

Saving Mapping.csv to Mapping (2).csv
✅ Patient data saved to /content/patients_unique.csv and /content/patients_unique.json



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Generated Clinical Notes (Preview):

--- Note for Patient 1 ---
Patient Name: Patient 1
Age: 55
Gender: Female
Symptoms: Headache, vision problems, nausea
Diagnosis: benign

Assessment: Based on the provided information, the patient presents with symptoms (Headache, vision problems, nausea) and imaging findings consistent with a benign condition. Further monitoring is recommended.

Plan: Schedule follow-up MRI in 6 months to monitor for changes. Advise on symptom management.
--------------------
--- Note for Patient 2 ---
Patient Name: Patient 2
Age: 41
Gender: Male
Symptoms: Severe headache, seizures, confusion
Diagnosis: malignant

Assessment: Based on the provided information, the patient presents with symptoms (Severe headache, seizures, confusion) highly suggestive of a malignant condition. Urgent investigation and specialist referral are warranted.

Plan: Refer to oncology for further evaluation and treatment planning. Order immediate CT scan.
--------------------
--- Note for 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Clinical notes saved to /content/clinical_notes.txt and downloaded.
